In [3]:
import os
import torch
import pinecone
from transformers import AutoModel, AutoTokenizer
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from tqdm import tqdm

c:\Users\vishw\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import pinecone
import torch
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from transformers import AutoModel, AutoTokenizer
import os

c:\Users\vishw\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
import os
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_ENV"] = "us-east-1"


In [5]:
# Initialize Pinecone
pc = pinecone.Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "sknai"

In [19]:
tokenizer = AutoTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")
model = AutoModel.from_pretrained("monologg/biobert_v1.1_pubmed")

def embed_text(text):
    """Generate BioBERT embeddings for a given text."""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().tolist() 

In [7]:
vector_store = Pinecone(
    index=pc.Index(index_name),  # Use the Pinecone client to access the index
    embedding=embed_text,        # Use the embedding function
    text_key="text"              # Assuming 'text' is the key for the documents
)

C:\Users\vishw\AppData\Local\Temp\ipykernel_14312\1600786067.py:1: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vector_store = Pinecone(
c:\Users\vishw\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [16]:
# Interactive query for similarity search
query = input("Enter your query for similarity search: ")
query_embedding = embed_text(query)

# Perform similarity search in Pinecone
pinecone_results = pc.Index(index_name).query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)

# Display the results from the similarity search
print("\nDirect Pinecone Query Results:")
for match in pinecone_results["matches"]:
    print("Score:", match["score"])
    print("Text:", match["metadata"].get("text", "No text available"))
    print("Chunk ID:", match.get("metadata", {}).get("chunk_id", "No chunk ID available"))
    print("-" * 50)


Direct Pinecone Query Results:
Score: 0.792724729
Text: dome. They usually resolve spontaneously 32.11 Warts within 6–18 months. They often become red and inflamed which is a sign that that individual See Chap. 34. lesion is regressing (Fig. 32.7). Treatment is usually not necessary unless they are very unsightly or uncomfortable. As 32.12 Molluscum Contagiosum molluscum are associated with dry skin, mois- turising the skin in the affected area with a thick, This is a harmless pox virus infection most com- grease moisturiser may help speed up the reso- monly seen in children with dry skin or eczema. lution. “Crystacide®” contains hydrogen perox- The pox virus may cause an immune response ide, which is known as a disinfectant and has resulting in
Chunk ID: 948
--------------------------------------------------
Score: 0.789101958
Text: lata is caused by secondary ent as tiny, white, dome-shaped papules with a syphilis. It can present as greyish-white moist central umbilicated crater on 

In [9]:
# Setup retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 50})

import getpass
import os

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")
    
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0.4,
    max_retries=2,
   
)


In [10]:
prompt = ChatPromptTemplate.from_messages([
    ("system", " You are a dermatologist and you are asked to provide answers to patient questions about skin conditions in very simple terms like a doctor who explains to the patient. You can use the following information to help you answer the questions:"),
    ("human", "{question}")
])
# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Stuffing retrieved docs into the prompt
    retriever=retriever,
    return_source_documents=True
)

In [11]:
from langchain_core.messages.tool import ToolOutputMixin


In [11]:
query = "What is bacterial infection?"
similar_docs = vector_store.similarity_search(query, k=3)  # top 3 similar documents

print("Retrieved Documents:")
for i, doc in enumerate(similar_docs, start=1):
    print(f"\nDocument {i}:")
    print("Content:", doc.page_content)  # or doc.content if that's your field name
    print("Metadata:", doc.metadata)

Retrieved Documents:

Document 1:
Content: gradually heals with routine care.One other rare complication only occurs with the largest ofcapillary angiomas.Blood platelets become sequestered in the abnor- mal vascular channels of the angioma,creating a consumption coagulopathy and uncontrolled bleeding (Katzenbach–Merritt syndrome).The bleeding can be dealt Figure 12.20 Small strawberry naevus on the with by administration ofsystemic steroids. left side of the nose. CAVERNOUS HAEMANGIOMA This is a soft, compressible, mauvish-blue
Metadata: {'chunk_id': '784'}

Document 2:
Content: skin is put together and how it functions in health.Although,at first glance,skin may appear quite complicated to the uniniti- ated, a slightly deeper look shows that there is a kind of elegant logic about its architecture,which is directed to subserving vital functions. THE SKIN SURFACE The skin surface is the delineation between living processes and the potentially injurious outside world and has not only a 

In [17]:
# Interactive loop for user queries
print("Welcome to the Interactive Dermatologist Assistant!")
print("Type 'exit' to quit.")

while True:
    query = input("\nYour question: ")
    
    if query.lower() == 'exit':
        print("Goodbye!")
        break
    
    # Test with the user query
    result = qa_chain({"query": query})

    # Print the AI response
    print("AI Response:", result["result"])
    
    # Option for follow-up questions
    follow_up = input("\nDo you have any follow-up questions? (yes/no): ")
    if follow_up.lower() == 'yes':
        continue
    else:
        print("Thank you for using the assistant!")
        break

Welcome to the Interactive Dermatologist Assistant!
Type 'exit' to quit.
AI Response: Based on the provided context, here are some treatment options available for warts:

1. **Salicylic Acid**: This is a topical treatment that has a cure rate of about 75% when used properly. It should be applied after paring down the hard keratin with an emery board or blade every night before applying the topical agent and continuing the treatment for at least 6–12 weeks.

2. **Cryotherapy**: This involves freezing the wart with liquid nitrogen. It can be effective but may require multiple treatments.

3. **Intralesional Bleomycin or Interferon**: These can be effective treatments for resistant warts but are expensive, very painful, and only available in some hospital dermatology departments.

4. **Duct Tape**: There is some evidence that covering the wart with duct tape for a period of time can help resolve it.

5. **Wait and See**: Warts may resolve on their own without treatment. They usually resol